In [107]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

In [108]:
print(tf.test.is_gpu_available())

True


In [109]:
def sanitize(string):
    words = string.split(' ')
    return words

def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split() if x not in stop_words])
    except AttributeError:
        return None  # or some other value
    
def whole(x):
    return int(round(x))

whole(1.0)

1

In [110]:
#read in the csv files 

quora = pd.read_csv("quora_duplicate_questions (1).tsv", sep='\t')


quora['question1']=quora['question1'].fillna("")
quora['question2']=quora['question2'].fillna("")


quora['title1_tokenized'] = \
    quora.loc[:, 'question1'] \
         .apply(sanitize)
quora['title2_tokenized'] = \
    quora.loc[:, 'question2'] \
         .apply(sanitize)



quora.head()

,id,qid1,qid2,question1,question2,is_duplicate,title1_tokenized,title2_tokenized
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[What, is, the, step, by, step, guide, to, inv...","[What, is, the, step, by, step, guide, to, inv..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[What, is, the, story, of, Kohinoor, (Koh-i-No...","[What, would, happen, if, the, Indian, governm..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[How, can, I, increase, the, speed, of, my, in...","[How, can, Internet, speed, be, increased, by,..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[Why, am, I, mentally, very, lonely?, How, can...","[Find, the, remainder, when, [math]23^{24}[/ma..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[Which, one, dissolve, in, water, quikly, suga...","[Which, fish, would, survive, in, salt, water?]"


In [44]:
Tb = Table()
list1 = []
list2 = []

with open("quora_duplicate_questions (1).tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(5):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        Tb=Tb.with_columns(list1[0][ele], list2[ele])


Tb.show(5)

id,qid1,qid2,question1,question2
0,1,2,What is the step by step guide to invest in share market ...,What is the step by step guide to invest in share market?
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Koh ...
2,5,6,How can I increase the speed of my internet connection w ...,How can Internet speed be increased by hacking through DNS?
3,7,8,Why am I mentally very lonely? How can I solve it?,Find the remainder when [math]23^{24}[/math] is divided ...
4,9,10,"Which one dissolve in water quikly sugar, salt, methane ...",Which fish would survive in salt water?


In [45]:
#create a dictionary

MAX_NUM_WORDS = 20000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

corpus_x1 = quora.title1_tokenized
corpus_x2 = quora.title2_tokenized
corpus = pd.concat([
  corpus_x1, corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(corpus)
x1 = tokenizer \
    .texts_to_sequences(corpus_x1)
x2 = tokenizer \
    .texts_to_sequences(corpus_x2)

word_index=tokenizer.word_index




corpus.head()

0    [What, is, the, step, by, step, guide, to, inv...
1    [What, is, the, story, of, Kohinoor, (Koh-i-No...
2    [How, can, I, increase, the, speed, of, my, in...
3    [Why, am, I, mentally, very, lonely?, How, can...
4    [Which, one, dissolve, in, water, quikly, suga...
dtype: object

In [46]:
max_seq_len = max([            #why does seq length change when I change dictionary size?
    len(seq) for seq in x1])
max_seq_len

115

In [47]:
max_seq_len1 = max([
    len(seq) for seq in x1])
print(max_seq_len1)   

MAX_SEQUENCE_LENGTH = 115  #better to have words covered than uncovered
x1 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2 = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1[:5]




115


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     2,     3,     1,  1394,    54,  1394,  2940,
            7,   519,     8,   748,   566,     8,    58],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

In [49]:


dup = quora.is_duplicate

y = keras \
    .utils \
    .to_categorical(dup)



print(len(quora))

y[:8]

404290


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [50]:
import math


def foo():
    q=[]
    p=0
    for x in quora.is_duplicate:
        if math.isnan(x):
            q = np.append(q, p)
        p+=1
    return q
foo()

[]

In [51]:
# We randomly select 5,000 paraphrases and 5,000 non-paraphrases as the dev set, and sample
#another 5,000 paraphrases and 5,000 non-paraphrases as the
#test set. We keep the remaining instances as the training set 2

label = quora.is_duplicate
eq_list = []
uneq_list=[]

count = 0

for x in range(len(quora.is_duplicate)):
    if label[x]==1:
        eq_list = np.append(eq_list, x) 
    
    else:
        uneq_list = np.append(uneq_list, x)

        
eq_list[:3]



array([ 5.,  7., 11.])

In [52]:
eq_list=eq_list.astype(int)
uneq_list=uneq_list.astype(int)

In [53]:
import random

random.shuffle(eq_list)
random.shuffle(uneq_list)

eq_list[:10]
print(len(eq_list))

149263


In [55]:
index_test_eq = eq_list[:5000]

index_test_uneq = uneq_list[:5000]

index_dev_eq = eq_list[5000:10000]

index_dev_uneq = uneq_list[5000:10000]

index_train_eq = eq_list[10000:]

index_train_uneq = uneq_list[10000:]

index_test = np.append(index_test_eq, index_test_uneq)

index_dev = np.append(index_dev_eq, index_dev_uneq)

index_train = np.append(index_train_eq, index_train_uneq)

random.shuffle(index_test)
random.shuffle(index_dev)
random.shuffle(index_train)



In [56]:


x1_test = [x1[i] for i in index_test]

x2_test = [x2[i] for i in index_test]

y_test = [y[i] for i in index_test]


x1_dev = [x1[i] for i in index_dev]

x2_dev = [x2[i] for i in index_dev]

y_dev=[y[i] for i in index_dev]


x1_train = [x1[i] for i in index_train]

x2_train = [x2[i] for i in index_train]

y_train=[y[i] for i in index_train]


print("Training Set")
print("-" * 10)
print(np.shape(x1_train))
print(np.shape(x2_train))
print(np.shape(y_train))


print("-" * 10)
print(np.shape(x1_test))
print(np.shape(x2_test))
print(np.shape(y_test))


print("Test Set")
print(y_test[:5])

Training Set
----------
(384290, 115)
(384290, 115)
(384290, 2)
----------
(10000, 115)
(10000, 115)
(10000, 2)
Test Set
[array([1., 0.], dtype=float32), array([1., 0.], dtype=float32), array([0., 1.], dtype=float32), array([0., 1.], dtype=float32), array([0., 1.], dtype=float32)]


In [67]:
np.array(y_test)[:10]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [96]:
#BiLSTM with no GLoVE or zero masking


NUM_CLASSES=2 #boolean; 1 or 0

MAX_NUM_WORDS = 20000 #rough estimate

MAX_SEQUENCE_LENGTH = 115 #how long one sentence is

NUM_EMBEDDING_DIM = 256 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')



embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 1024

NUM_EPOCHS = 15

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min')
mcp_save = ModelCheckpoint('quora_weights.h5', save_best_only=True, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 115)          0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 115)          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 115, 256)     5120000     input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 256)          394240      embedding_8[0][0]                
          

In [97]:

history = model.fit(x=([np.array(x1_train), np.array(x2_train)]), y=np.array(y_train), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping, mcp_save], validation_data=([np.array(x1_dev), np.array(x2_dev)], np.array(y_dev)),
    
    shuffle=True
)

Train on 384290 samples, validate on 10000 samples
Epoch 1/15
384290/384290 [==============================] - 334s 868us/step - loss: 0.5179 - acc: 0.7464 - val_loss: 0.5366 - val_acc: 0.7308
Epoch 2/15
384290/384290 [==============================] - 308s 802us/step - loss: 0.4519 - acc: 0.7877 - val_loss: 0.5317 - val_acc: 0.7409
Epoch 3/15
384290/384290 [==============================] - 306s 795us/step - loss: 0.4139 - acc: 0.8077 - val_loss: 0.5069 - val_acc: 0.7532
Epoch 4/15
384290/384290 [==============================] - 305s 793us/step - loss: 0.3816 - acc: 0.8232 - val_loss: 0.5274 - val_acc: 0.7607
Epoch 5/15
384290/384290 [==============================] - 313s 815us/step - loss: 0.3504 - acc: 0.8373 - val_loss: 0.5719 - val_acc: 0.7584
Epoch 6/15
384290/384290 [==============================] - 306s 795us/step - loss: 0.3189 - acc: 0.8495 - val_loss: 0.5986 - val_acc: 0.7558


In [ ]:
model = load_model("BiLSTM_QQP.h5")



In [112]:
predicts = model.predict(
    [x1_train, x2_train])

array([1., 0.], dtype=float32)

In [ ]:

no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(predicts)):
    if (np.argmax(predicts[x])==(np.argmax(y_train[x]))):
        no=np.append(no, x)
        err1=np.append(err1, x1_train[x])
        err2=np.append(err2, x2_train[x])
        wrongCat = np.append(wrongCat, y_train[x])
        
print(1-(len(wrongCat)/len(x1_train)))

In [ ]:
error_analysis = Table()


error_analysis=error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "classification", wrongCat)
error_analysis
           


In [ ]:
hue = error_analysis.where("classification", are.equal_to(0)) 

In [39]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

NUM_EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index)+1, NUM_EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [111]:
model = load_model('BiLSTM_QQP.h5')